# Check GPU options

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Import needed libraries

In [ ]:
# Import general purpose python libraries
import os
import matplotlib.pyplot as plt
from PIL import Image # For handling the images
import numpy as np
from sklearn import metrics
import pandas as pd
# Import different Keras functionalities
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
from keras.utils import np_utils
#from tensorflow.keras.optimizers import SGD,Adagrad
from keras.optimizers import SGD,Adagrad
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras import backend as K
#from keras.applications import ResNet50,Xception,VGG16,VGG19
from sklearn.model_selection import train_test_split

from plotcm import plotcm

####
# Where to save the figures
IMAGES_PATH = "../images"
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
from utils import saveFigures,write_json,create_json,load_previous_weights

from models import VGG16, MLP_model

# Load data

In [ ]:
cifar10 = keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=1)

X_train = X_train/255.
X_valid = X_valid/255.
X_test = X_test/255.

y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

print("num_training_samples: ", X_train.shape)

# Configuration parameters
Configuration values of different parts of the solution. You should change some of them to obtain better results.

In [ ]:
from datetime import datetime
# Randomize the initial network 
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


# Directory where to store weights of the model and results
now = datetime.now()
current_time = now.strftime("%d_%H_%M_%S")
experiment_rootdir = "./test/"+current_time+"/"
weights_path = experiment_rootdir +"weights.h5"
weights_best_path = experiment_rootdir +"weights_best.h5"
json_path = experiment_rootdir +"experiment.json"

# Create experiment directory if it does not exists
if not os.path.exists(experiment_rootdir):
    os.makedirs(experiment_rootdir)
print(experiment_rootdir)

In [ ]:
# Parameters that configures the training process

# Tunable parameters
name_model = "MLP-10layers"
initial_lr = 5e-2

l2_reg = 0.0
dropout = 0.0

compile_parameters = { "metrics": ['accuracy'], "optimizer": SGD(learning_rate=initial_lr)}

# TODO


## Constant parameters
batch_size = 512 
epochs = 200
initial_epoch = 0 
num_classes = 10
dropout = 0.5

checkpoint_cb = ModelCheckpoint(weights_best_path,save_best_only=True)
early_stopping_cb = EarlyStopping(patience=25,restore_best_weights=True)
'''
optimizer=  = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
'''

parameters = {
    "name_model":name_model,
    "compile_parameters" :str(compile_parameters) ,
    "batch_size": batch_size,
    "epochs": epochs, 
    "initial_epoch":initial_epoch, 
    "initial_lr": initial_lr,
    "num_classes": num_classes,
    "dropout": dropout

}
create_json({"parameters":parameters}, json_path)


In [ ]:
def trainModel(model,compile_parameters,X_train, y_train, validation_data, epochs, batch_size, experiment_rootdir, weights_path, json_path):
    model.compile(loss='categorical_crossentropy', optimizer=compile_parameters['optimizer'], metrics=compile_parameters['metrics'])
    print(model.summary())
    history = model.fit(x= X_train, y= y_train, validation_data = validation_data, epochs = epochs, batch_size= batch_size,callbacks=[checkpoint_cb, early_stopping_cb])
    model.save_weights(weights_path)
    hist_df = pd.DataFrame(history.history) 
    data = {"train_result2":hist_df.to_dict()}
    write_json(data,json_path)
    return history

# Training 

In [ ]:
#Get model
model = MLP_model(n_hidden=10, num_classes=10, img_width=32, img_height=32, img_channels=3,l2_reg=0)
#model = VGG16(dropout=0.5, num_classes=10, img_width=32, img_height=32, img_channels=3,l2_reg=0)

#load weights
load_previous_weights(model,weights_path)

#training model
val = (X_valid, y_valid)

history =trainModel(model,compile_parameters,X_train, y_train,val, epochs, batch_size,experiment_rootdir,weights_path,json_path)

saveFigures(experiment_rootdir, history, "accuracy", "loss")

# Testing

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
gt = np.argmax(y_test, axis=1)

# Evaluate predictions: Average accuracy and highest errors
print("-----------------------------------------------")
print("Evaluation:")
# Compute average accuracy
ave_accuracy = metrics.accuracy_score(gt, y_pred)
print('Average accuracy = ', ave_accuracy)
print("-----------------------------------------------")

# Visualize confusion matrix                                           
plotcm(experiment_rootdir, gt, y_pred,class_names, experiment_rootdir, normalize=True)

data = {"test_accuracy":ave_accuracy}
write_json(data,json_path)

